# Random Forest
Cold nuclei from botton quintile.  
Run on Windows / Precision.
Train on 5-class cancer classification.  
Input is the nucleus+RBC rollup, training set, cold nuclei only.    
Run 5-fold cross validation.  
Print the mean accuracy. 

The Image.csv is a copy of the original.  
The RBC_rollup.csv is a copy of the original.  
The Nucleus_rollup.csv summarizes only the low-temperature nuclei.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-11 13:43:08.732451
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='D:\\Adjeroh\\Naved\\CP_80K_ColdQuint\\'  # append Output0/ etc
CLASSES=range(0,6)  # use all 6 classes
CLASSES=range(1,6)  # use just 5 classes
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.24'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = cp.get_RBC_rollup()
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-11 13:43:12.357143
2022-06-11 13:43:12.357143
Process class 1
Num WSI in test/train sets: 7 26
Num patches in test/train sets: 2734 10168
2022-06-11 13:43:30.565878
Process class 2
Num WSI in test/train sets: 7 30
Num patches in test/train sets: 2769 11181
2022-06-11 13:43:50.849259
Process class 3
Num WSI in test/train sets: 3 14
Num patches in test/train sets: 1091 5273
2022-06-11 13:44:00.725234
Process class 4
Num WSI in test/train sets: 2 6
Num patches in test/train sets: 813 2384
2022-06-11 13:44:05.539257
Process class 5
Num WSI in test/train sets: 1 3
Num patches in test/train sets: 396 1195


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_BoundingBoxArea_count,Nuc_AreaShape_BoundingBoxArea_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
1209,12.0,502.500000,247.269636,228.0,290.00,455.5,710.00,882.0,12.0,754.916667,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1210,8.0,391.000000,179.332094,182.0,243.25,368.5,560.25,607.0,8.0,677.875000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1211,22.0,566.272727,285.284016,186.0,366.25,532.5,721.00,1146.0,22.0,944.545455,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1212,10.0,470.600000,124.351831,307.0,368.50,448.5,581.75,649.0,10.0,765.800000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1213,11.0,389.000000,112.305832,187.0,301.00,438.0,463.50,527.0,11.0,622.818182,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,3.0,306.000000,112.529996,177.0,267.00,357.0,370.50,384.0,3.0,456.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,8.0,518.000000,325.028350,188.0,345.00,439.5,592.25,1190.0,8.0,892.625000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,9.0,347.444444,133.153212,191.0,286.00,320.0,378.00,618.0,9.0,494.000000,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-11 13:44:09.246956
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Cross-validation...')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-11 13:44:09.855031
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 2.3min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV] END .................................................... total time= 2.3min
[CV] END .................................................... total time= 2.3min
[CV] END .................................................... total time= 2.5min
[CV] END .................................................... total time= 2.4min
[0.6445722  0.63943853 0.65482834 0.65668865 0.63960764]
mean 0.6470 +/- 0.0074


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 11.8min finished


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_model(),MODELS_DIR)

2022-06-11 13:55:57.707463


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.22']